### Gives the description of an image

In [1]:
import warnings,logging
def cap(image_url):
    warnings.simplefilter('ignore')
    logging.disable(logging.WARNING)
    from transformers import pipeline
    caption = pipeline('image-to-text')
    from IPython.display import Image, display

    # URL of the image

    # Display the image
    display(Image(url=image_url))
    text=caption(image_url)
    return text[0]['generated_text']

### To convert Text into Braille

In [2]:
def TextToBraille(text):
    braille_map = {
        'a': '⠁',
        'b': '⠃',
        'c': '⠉',
        'd': '⠙',
        'e': '⠑',
        'f': '⠋',
        'g': '⠛',
        'h': '⠓',
        'i': '⠊',
        'j': '⠚',
        'k': '⠅',
        'l': '⠇',
        'm': '⠍',
        'n': '⠝',
        'o': '⠕',
        'p': '⠏',
        'q': '⠟',
        'r': '⠗',
        's': '⠎',
        't': '⠞',
        'u': '⠥',
        'v': '⠧',
        'w': '⠺',
        'x': '⠭',
        'y': '⠽',
        'z': '⠵',
        '1': '⠼⠁',
        '2': '⠼⠃',
        '3': '⠼⠉',
        '4': '⠼⠙',
        '5': '⠼⠑',
        '6': '⠼⠋',
        '7': '⠼⠛',
        '8': '⠼⠓',
        '9': '⠼⠊',
        '0': '⠼⠚',
        '!': '⠖',
        '(': '⠶',
        ')': '⠶',
        '-': '⠤',
        '{': '⠸⠜',
        '[': '⠶⠠',
        ']': '⠶⠠',
        '}': '⠸⠜',
        ':': '⠒',
        ';': '⠆',
        ',': '⠂',
        '.': '⠲',
        '/': '⠌',
        '?': '⠦',
        ' ': ' ',
        '\n': '\n'
    }
    final_string = ""
    for char in text:
        char = char.lower()
        if char in braille_map:
            final_string += braille_map[char]
        else:
            continue
    return final_string

### Using Keras model to do OCR on image

In [3]:
import keras_ocr
import matplotlib.pyplot as plt
def anotate(image_path):
    # Create a pipeline
    pipeline = keras_ocr.pipeline.Pipeline()
    

    # Get an image
#     image_path = 'C://Users//jayag//Desktop//Marwadi//SEM-6//Mini-Project//3.jpeg'
    image = keras_ocr.tools.read(image_path)
    

    # Use the pipeline to extract text from the image
    prediction_groups = pipeline.recognize([image])

    # Plot the predictions
    fig, ax = plt.subplots()
    keras_ocr.tools.drawAnnotations(image=image, predictions=prediction_groups[0], ax=ax)
    plt.savefig("static/result.jpg")
    
        # Extract Predictions
    prediction_groups = pipeline.recognize([image])

    # Group words by their y-coordinate to identify lines of text
    line_groups = {}
    for word_group in prediction_groups[0]:
        y_coordinate = round(word_group[1][0][1], 2)  # Round y-coordinate to handle floating point errors
        if y_coordinate in line_groups:
            line_groups[y_coordinate].append(word_group)
        else:
            line_groups[y_coordinate] = [word_group]

    # Sort the lines based on their y-coordinate
    sorted_lines = sorted(line_groups.items(), key=lambda item: item[0])

    # Extract words from sorted lines and combine them to form the final sequence
    detected_sequence = ''
    for _, line in sorted_lines:
        # Sort words within each line based on their x-coordinate
        sorted_words = sorted(line, key=lambda word_group: word_group[1][0][0])
        line_words = [word_group[0] for word_group in sorted_words]

        # Check text direction
        if len(line_words) >= 2 and sorted_words[0][1][0][0] > sorted_words[1][1][0][0]:
            # If the text is detected from right to left, reverse the order of words within the line
            line_words.reverse()

        line_text = ' '.join(line_words)
        detected_sequence += line_text + ' '

    # Print the final sequence
    print("Detected Sequence:", detected_sequence)
    return detected_sequence


### Main flask app

In [ ]:
from flask import Flask, render_template, request
from PIL import Image
import pytesseract
from gtts import gTTS
# from ultralytics import YOLO
import cv2
# import cvzone
import math
import numpy as np

import shutil
import os


app = Flask(__name__)



@app.route('/')
def index():
    return render_template('index.html')

@app.route('/image_to_text', methods=['GET', 'POST'])
def image_to_text():
    if request.method == 'POST':
        if 'image' not in request.files:
            return "No image found in request!"
        image = request.files['image']
        if image.filename == '':
            return "No image selected!"
        try:
            result = anotate(image)
            result=TextToBraille(result)
            return render_template('image_to_text.html', result=result,path=image)
        except Exception as e:
            return f"An error occurred: {str(e)}"
    else:
        return render_template('image_to_text.html')
    
    
@app.route('/image_to_speech', methods=['GET', 'POST'])
def image_to_speech():
    result=0
    if request.method == 'POST':
        if 'image' not in request.files:
            return "No image found in request!"

        image = request.files['image']
        if image.filename == '':
            return "No image selected!"

        try:
            img = Image.open(image)
            text = pytesseract.image_to_string(img)
#             text = anotate(image)
            

            # Convert text to speech
            tts = gTTS(text=text, lang='en')
            tts.save("static/output.mp3")
            result=1
            # Return the path to the generated speech file
            return render_template('image_to_speech.html',result=result,text=text)
        except Exception as e:
            return f"An error occurred: {str(e)}"
    else:
        return render_template('image_to_speech.html')
    
@app.route('/object_detection', methods=['GET', 'POST'])
def object_detection():
    if request.method == 'POST':
        url = request.form['url']
        try:
            result = cap(url)
            return render_template('object_detection.html', result=result)
        except Exception as e:
            return f"An error occurred: {str(e)}"
    else:
        return render_template('object_detection.html')
    
if __name__ == '__main__':
    app.run(debug=True,use_reloader=False)


 * Serving Flask app '__main__'
 * Debug mode: on


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
127.0.0.1 - - [06/Jul/2024 17:11:29] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [06/Jul/2024 17:11:29] "GET /favicon.ico HTTP/1.1" 404 -
127.0.0.1 - - [06/Jul/2024 17:11:33] "GET /image_to_text HTTP/1.1" 200 -


Looking for C:\Users\jayag\.keras-ocr\craft_mlt_25k.h5



127.0.0.1 - - [06/Jul/2024 17:11:43] "POST /image_to_text HTTP/1.1" 200 -
